In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
att_close = pd.read_csv("PA_close_new.csv")
att_open = pd.read_csv("PA_open_new.csv")
print(att_close.shape)
print(att_open.shape)

(226, 57)
(279, 58)


In [3]:
li = ['business_id', 'latitude', 'longitude', 'RestaurantsPriceRange2']
new_close = att_close.loc[:,li]
new_open = att_open.loc[:,li]

In [4]:
df_combined = pd.concat([new_close, new_open])
df_combined.shape

(505, 4)

In [5]:
longitude_1 = 39.92
longitude_2 = 39.97
latitude_1 = -75.22
latitude_2 = -75.11


In [6]:
np.unique(att_close.RestaurantsPriceRange2)

array([ 1.,  2.,  3.,  4., nan])

In [7]:
center_high = []
center_low = []
nocen_high = []
nocen_low = []
df_combined = df_combined.dropna(subset=['RestaurantsPriceRange2'])
for i in range(np.shape(df_combined)[0]):
    lo = df_combined.iloc[i,2]
    la = df_combined.iloc[i,1]
    if (la > longitude_1 and la < longitude_2  and lo > latitude_1 and lo < latitude_2):
        if  df_combined.iloc[i,3] >2.5:
            center_high.append(df_combined.iloc[i,0])
        else:
            center_low.append(df_combined.iloc[i,0])
            
    else:
        if  df_combined.iloc[i,3] >2.5:
            nocen_high.append(df_combined.iloc[i,0])
        else:
            nocen_low.append(df_combined.iloc[i,0])

In [8]:
import json
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from collections import Counter

In [9]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rzjin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rzjin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
data_re= []

with open('review.json', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        data_re.append(data)

review = pd.DataFrame(data_re)

In [11]:
review_business_id = review["business_id"]
review_ch = []
review_cl = []
review_nh = []
review_nl = []



for i in range(len(review_business_id)):
    if review_business_id[i] in center_high:
        review_ch.append(i)
    elif review_business_id[i] in center_low:
        review_cl.append(i)
    elif review_business_id[i] in nocen_high:
        review_nh.append(i)
    elif review_business_id[i] in nocen_low:
        review_nl.append(i)

In [12]:
PA_ch = review.iloc[review_ch,]
PA_cl = review.iloc[review_cl,]
PA_nh = review.iloc[review_nh,]
PA_nl = review.iloc[review_nl,]

In [13]:
service_words = ["attitude", "service", "staff", "waiter", "manager", "host", "friendly", "waitress", "attentive"]
location_words = ["convenient","location","nearby","parking","central","accessible","remote","takeout"]
environment_words = ["atmosphere","decor","clean","environment","ambiance","noisy","spacious","quiet","cozy"]
hygiene_safety_words = ["safe","sanitary","untidy","secure","hygienic","sterile","orderly","regulated"]
price_value_words = ["budget","worth","overpriced","expensive","cheap","price","deal","affordable","value"]

In [54]:
csvfile = PA_nl

In [55]:
csvfile['service'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in service_words))
csvfile['location'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in location_words))
csvfile['environment'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in environment_words))
csvfile['hygiene_safety'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in hygiene_safety_words))
csvfile['price_value'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in price_value_words))

C:\Users\rzjin\AppData\Local\Temp\ipykernel_66932\1605387618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csvfile['service'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in service_words))
C:\Users\rzjin\AppData\Local\Temp\ipykernel_66932\1605387618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csvfile['location'] = csvfile['text'].str.lower().apply(lambda x: any(word in x for word in location_words))
C:\Users\rzjin\AppData\Local\Temp\ipykernel_66932\1605387618.py:3: Set

In [56]:
index_count = csvfile.groupby('stars')[['service','location','environment','hygiene_safety','price_value']].sum()

In [57]:
review_count = csvfile.groupby('stars').size()

In [58]:
index_count['review_count']=review_count

In [59]:
index_count

,service,location,environment,hygiene_safety,price_value,review_count
stars,,,,,,
1.0,755,153,106,28,364,1707
2.0,437,112,85,11,283,969
3.0,522,179,175,11,380,1219
4.0,1082,295,404,13,810,2515
5.0,2026,374,641,43,1093,4473


In [60]:
index_count.to_csv('PA_nl_df.csv', index = True)

## Dishes Count

In [61]:
dishes = ["pizza", "cheese", "dessert", "chicken", "salad", "pasta", "wine", "soup", "seafood", "beef"]

In [62]:
csvfile['dishes'] = csvfile['text'].str.lower().str.split().apply(lambda x: [word for word in x if word in dishes])

C:\Users\rzjin\AppData\Local\Temp\ipykernel_66932\864959041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csvfile['dishes'] = csvfile['text'].str.lower().str.split().apply(lambda x: [word for word in x if word in dishes])


In [63]:
dishes_count = csvfile.groupby('stars')['dishes'].apply(lambda x: x.explode().value_counts()).unstack(fill_value=0)

In [64]:
dishes_count['review_count']=review_count

In [65]:
dishes_count

,beef,cheese,chicken,dessert,pasta,pizza,salad,seafood,soup,wine,review_count
stars,,,,,,,,,,,
1.0,21,273,267,11,54,674,144,35,52,19,1707
2.0,14,198,227,17,58,463,120,12,22,28,969
3.0,24,252,240,27,54,666,145,34,43,25,1219
4.0,61,417,460,66,153,1355,255,101,99,89,2515
5.0,65,516,643,85,253,2081,294,216,104,125,4473


In [66]:
dishes_count.to_csv('PA_nl_dishes.csv', index = True)